In [ ]:
import hyperspy.api as hs
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import subprocess

script_path = "/~/correct_256.py"

base_path = '/~/Merlin/subfolder'
dir_paths = glob.glob(base_path+"/*")
dir_paths.sort()
print(*dir_paths, sep="\n")

In [ ]:
bash_script_path = os.path.join(base_path, 'cluster_submit.sh')
with open(bash_script_path, 'w') as f:
    f.write("#!/usr/bin/env bash\n")
    f.write("#SBATCH --partition=cs04r\n")
    f.write("#SBATCH --job-name=hspy_change\n")
    f.write("#SBATCH --nodes=1\n")
    f.write("#SBATCH --ntasks-per-node=4\n")
    f.write("#SBATCH --cpus-per-task=1\n")
    f.write("#SBATCH --time=12:00:00\n")
    f.write("#SBATCH --mem=0G\n")
    f.write("#SBATCH --output=%s/%%j.out\n"%base_path)
    f.write("#SBATCH --error=%s/%%j.error\n\n"%base_path)
    
    f.write("module load python/epsic3.10\n")
    f.write(f'python {script_path} {base_path}')

sshProcess = subprocess.Popen(['ssh',
                               '-tt',
                               'wilson'],
                               stdin=subprocess.PIPE, 
                               stdout = subprocess.PIPE,
                               universal_newlines=True,
                               bufsize=0)
sshProcess.stdin.write("echo END\n")
sshProcess.stdin.write("sbatch "+bash_script_path+"\n")
sshProcess.stdin.write("uptime\n")
sshProcess.stdin.write("logout\n")
sshProcess.stdin.close()